# Базовые понятия статистики

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(
    'horse_data.csv', usecols = [0,1,3,4,5,6,10,22],
    names = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'],
    na_values = "?")
df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


### выбросы по столбцам

- функция, далее применение её к каждому столбцу 

In [15]:
def outline(df, col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    
    iqr = q3 - q1
    
    lower_bound = q1 -(1.5 * iqr) 
    upper_bound = q3 +(1.5 * iqr) 
    out = df[(col < lower_bound)|(col > upper_bound)]
    return out

In [17]:
outline(df, df['rectal temperature'])

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
54,2.0,1,40.3,114.0,36.0,3.0,2.0,3.0
75,1.0,9,39.7,100.0,NaN,3.0,2.0,3.0
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0
91,2.0,1,40.3,114.0,36.0,3.0,2.0,2.0
99,2.0,1,39.6,108.0,51.0,3.0,2.0,1.0
118,1.0,1,36.5,78.0,30.0,1.0,5.0,1.0
141,2.0,1,36.0,42.0,30.0,NaN,NaN,2.0
238,2.0,1,36.1,88.0,NaN,3.0,3.0,3.0


In [16]:
outline(df, df['pulse'])

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,9,38.6,160.0,20.0,3.0,3.0,2.0
255,1.0,9,38.8,184.0,84.0,1.0,4.0,2.0
275,1.0,9,38.8,150.0,50.0,1.0,5.0,2.0


In [18]:
outline(df, df['respiratory rate'])

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
39,1.0,9,39.2,146.0,96.0,NaN,NaN,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
82,1.0,9,38.1,100.0,80.0,3.0,3.0,1.0
84,1.0,1,37.8,60.0,80.0,1.0,2.0,1.0
103,1.0,9,38.0,140.0,68.0,1.0,3.0,1.0
106,1.0,1,38.3,52.0,96.0,NaN,NaN,1.0
120,1.0,1,39.4,54.0,66.0,1.0,2.0,1.0
125,1.0,1,38.0,42.0,68.0,4.0,3.0,1.0
186,1.0,1,39.3,64.0,90.0,2.0,NaN,1.0


In [20]:
outline(df, df['temperature of extremities'])

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome


In [21]:
outline(df, df['pain'])

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome


### базовые статистики

In [287]:
cols = df.columns 

for i in range(len(cols)):
	print(f'Столбец "{cols[i]}"\n\
    Минимальное значение - {df[cols[i]].min()}\n\
    Максимальное значение - {df[cols[i]].max()}\n\
    Медиана - {df[cols[i]].median()}\n\
    Среднее арифметическое - {df[cols[i]].mean()}\n\
    Мода - {df[cols[i]].round().mode()[0]}\n')


Столбец "surgery?"
    Минимальное значение - 1.0
    Максимальное значение - 2.0
    Медиана - 1.0
    Среднее арифметическое - 1.3979933110367893
    Мода - 1.0

Столбец "Age"
    Минимальное значение - 1
    Максимальное значение - 9
    Медиана - 1.0
    Среднее арифметическое - 1.64
    Мода - 1

Столбец "rectal temperature"
    Минимальное значение - 35.4
    Максимальное значение - 40.8
    Медиана - 38.2
    Среднее арифметическое - 38.16791666666669
    Мода - 38.0

Столбец "pulse"
    Минимальное значение - 30.0
    Максимальное значение - 184.0
    Медиана - 64.0
    Среднее арифметическое - 71.91304347826087
    Мода - 48.0

Столбец "respiratory rate"
    Минимальное значение - 8.0
    Максимальное значение - 96.0
    Медиана - 24.5
    Среднее арифметическое - 30.417355371900825
    Мода - 20.0

Столбец "temperature of extremities"
    Минимальное значение - 1.0
    Максимальное значение - 4.0
    Медиана - 3.0
    Среднее арифметическое - 2.348360655737705
    Мода - 3.0


# заполнения пропусков 
### что заполняем  - В зависимости от чего, предположение
 - Ректальная температура - возраст 
 - пульс - возраст, температура ректальная 
 - Боль - хирургия 
 - Дыхание - боль 
 - температура конечностей - возраст, дыхание 

In [345]:
df['rectal temperature'] = df['rectal temperature'].fillna(df.groupby('Age')['rectal temperature'].transform('median'))  
df['pulse'] = df['pulse'].fillna(df.groupby(['Age', 'rectal temperature'])['pulse'].transform('median'))
df['pain'] = df['pain'].fillna(df.groupby('surgery?')['pain'].transform('max'))  
df['respiratory rate'] = df['respiratory rate'].fillna(df.groupby('pain')['respiratory rate'].transform('median'))  
df['temperature of extremities'] = df['temperature of extremities'].fillna(df.groupby(['respiratory rate', 'Age'])['temperature of extremities'].transform('median'))


df.head(10)

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,2.0,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,3.5,5.0,2.0
5,2.0,1,38.1,70.0,32.0,2.0,2.0,1.0
6,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
7,1.0,1,38.1,60.0,28.0,3.0,5.0,2.0
8,2.0,1,38.1,80.0,36.0,3.0,4.0,3.0
9,2.0,9,38.3,90.0,28.0,1.0,5.0,1.0
